In [5]:
import pandas as pd 
data =  pd.read_csv('SMSSpamCollection', sep = '\t', names = ['label', 'message'])

In [7]:
text = data['message']
label = data['label']

In [8]:
data['word_count'] = data['message'].apply(lambda x: len(str(x).split(" ")))

In [9]:
data['char_count'] = data['message'].str.len() ## this also includes spaces
data[['message','char_count']].head()

,message,char_count
0,"Go until jurong point, crazy.. Available only ...",111
1,Ok lar... Joking wif u oni...,29
2,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,U dun say so early hor... U c already then say...,49
4,"Nah I don't think he goes to usf, he lives aro...",61


In [10]:
def avg_word(sentence):
  words = sentence.split()
  #print(words)
  return (sum(len(word) for word in words)/len(words))

data['avg_word'] = data['message'].apply(lambda x: avg_word(x))

In [11]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')

data['stopwords'] = data['message'].apply(lambda x: len([x for x in x.split() if x in stop]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
data['hastags'] = data['message'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))

In [13]:
data['numerics'] = data['message'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))

In [14]:
data['upper'] = data['message'].apply(lambda x: len([x for x in x.split() if x.isupper()]))

In [15]:
pos_family = {
    'noun' : ['NN','NNS','NNP','NNPS'],
    'pron' : ['PRP','PRP$','WP','WP$'],
    'verb' : ['VB','VBD','VBG','VBN','VBP','VBZ'],
    'adj' :  ['JJ','JJR','JJS'],
    'adv' : ['RB','RBR','RBS','WRB']
}

# function to check and get the part of speech tag count of a words in a given sentence
from textblob import TextBlob, Word, Blobber
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
def check_pos_tag(x, flag):
    cnt = 0
    try:
        wiki = TextBlob(x)
        for tup in wiki.tags:
            ppo = list(tup)[1]
            if ppo in pos_family[flag]:
                cnt += 1
    except:
        pass
    return cnt

data['noun_count'] = data['message'].apply(lambda x: check_pos_tag(x, 'noun'))
data['verb_count'] = data['message'].apply(lambda x: check_pos_tag(x, 'verb'))
data['adj_count'] = data['message'].apply(lambda x: check_pos_tag(x, 'adj'))
data['adv_count'] = data['message'].apply(lambda x: check_pos_tag(x, 'adv'))
data['pron_count'] = data['message'].apply(lambda x: check_pos_tag(x, 'pron'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [22]:
features = data[['word_count','char_count','avg_word','stopwords','hastags','numerics','upper','noun_count','verb_count','adj_count', 'adv_count', 'pron_count']]

In [24]:
import numpy as np
features_array = np.asarray(features)

Kmeans

In [25]:
from sklearn.cluster import KMeans

In [26]:
k=2
model=KMeans(n_clusters=k)
model.fit(features_array)
clusassign=model.predict(features_array)

In [27]:
from sklearn.metrics import accuracy_score
from sklearn import metrics


print("k-means")
print("Accuracy score =", accuracy_score(label, clusassign))
print(metrics.confusion_matrix(label,clusassign))

print(metrics.classification_report(label, clusassign))

k-means
Accuracy score = 0.7873295046661881
[[3715 1110]
 [  75  672]]
              precision    recall  f1-score   support

           0       0.98      0.77      0.86      4825
           1       0.38      0.90      0.53       747

    accuracy                           0.79      5572
   macro avg       0.68      0.83      0.70      5572
weighted avg       0.90      0.79      0.82      5572

